<a href="https://colab.research.google.com/github/eeee-day/hongong_data/blob/main/(code)%20%EB%B6%80%EB%A1%9DA_%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%97%90%EC%84%9C%20SQL%20%EC%82%AC%EC%9A%A9%ED%95%98%EA%B8%B0-%EC%9D%B4%EB%A1%A0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#파이썬 임베디드 DB SQLite 사용하기

import sqlite3

> ⭐ 코드를 실행하며 발생한 문제점

* INSERT 쿼리문에서 OperationalError: unrecognized token: "\" 오류 발생

In [2]:
#데이터베이스 연결하기 : connect()함수

conn = sqlite3.connect('ns_lib.db')

In [3]:
import gdown
gdown.download('https://bit.ly/3RhoNho', 'ns_202104.csv', quiet=False)

Downloading...
From: https://bit.ly/3RhoNho
To: /content/ns_202104.csv
100%|██████████| 57.6M/57.6M [00:01<00:00, 41.8MB/s]


'ns_202104.csv'

In [4]:
#테이블 만들기 : CREATE TABLE 문
#cursor 객체 만들기 (*DB의 데이터를 쿼리하여 가져올 때 사용)
#execute() 메서드를 통해 SQL문 전달

c = conn.cursor()
c.execute("CREATE TABLE IF NOT EXISTS nslib_book(name TEXT, author TEXT, borrow_count INTEGER)")


In [5]:
#데이터프레임 데이터를 테이블에 추가하기

import pandas as pd
ns_df = pd.read_csv('ns_202104.csv',low_memory=False) #low_memory=False : 경고메세지 출려 x
ns_df.head()

,번호,도서명,저자,출판사,발행년도,ISBN,세트 ISBN,부가기호,권,주제분류번호,도서권수,대출건수,등록일자,Unnamed: 13
0,1,인공지능과 흙,김동훈 지음,민음사,2021,9788937444319,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
1,2,가짜 행복 권하는 사회,김태형 지음,갈매나무,2021,9791190123969,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
2,3,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,블랙피쉬,2021,9788968332982,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
3,4,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",문학세계사,2021,9788970759906,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN
4,5,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,김영사,2021,9788934990833,NaN,NaN,NaN,NaN,1,0,2021-03-19,NaN


In [6]:
# 한 행씩 반복 실행하기 : for문과 iterrows() 메서드

c.execute('INSERT INTO nslib_book (name, author, borrow_count) VALUES(?,?,?)', ('인공지능과 흙','김동훈 지음','0'))

In [7]:
for index, row in ns_df.iterrows():
  c.execute("INSERT INTO nslib_book (name, author,borrow_count) VALUES(?,?,?)",(row['도서명'], row['저자'],row['대출건수']))

In [8]:
#SQL문 없이 DF 데이터를 테이블에 추가하기 : to_sql() 메서드

book_df = ns_df[['도서명','저자','대출건수']]
book_df.head()

,도서명,저자,대출건수
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [9]:
book_df.columns = ['name','author','borrow_count'] #columns를 사용하여 간단하게 열 이름 변경 가능
book_df.head()

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0


In [10]:
book_df.to_sql('nslib_book',conn,if_exists='replace',index=False)

401682

> if_exists 매개변수 옵션

* fail : 기본값, 첫 번째 매개변수에 지정된 테이블이 이미 있으면 오류 발생

* replace : 테이블을 지우고 새로 만든 다음 데이터를 추가

* append : 기존의 테이블에 데이터를 추가

In [27]:
#p.448
#파이썬으로 테이블에서 데이터 읽기
c.execute("SELECT * FROM nslib_book")

In [28]:
#위에서 선택한 테이블 확인
c.fetchone()
#fetchone : 지정한 열 이름 순서대로 데이터가 나열된 파이썬 튜플 반환.

('인공지능과 흙', '김동훈 지음', 0)

In [29]:
c.fetchone()
#두 번 호출하면 그 다음 열을 순서대로 반환

('가짜 행복 권하는 사회', '김태형 지음', 0)

In [30]:
c.fetchmany(3)
#1개 이상의 행을 호출할 때 사용

[('나도 한 문장 잘 쓰면 바랄 게 없겠네', '김선영 지음', 0),
 ('예루살렘 해변', '이도 게펜 지음, 임재희 옮김', 0),
 ('김성곤의 중국한시기행 : 장강·황하 편', '김성곤 지음', 0)]

In [31]:
all_rows = c.fetchall()
#모든 행을 선택할 때 사용

In [32]:
book_df = pd.DataFrame(all_rows)
book_df.head()

,0,1,2
0,처음 읽는 음식의 세계사,"미야자키 마사카츠 지음, 한세희 옮김",0
1,아르센 벵거 자서전 My Life in Red and White,"아르센 벵거 지음, 이성모 옮김",0
2,쓰고 달콤한 직업,천운영 지음,0
3,일잘러의 노션 100% 활용법,제레미 강 지음,0
4,타이난 골목 노포 산책,"천구이팡 지음, 심혜경 외 옮김",0


In [33]:
#더 간편하게 데이터프레임으로 읽기 : read_sql_query()함수

book_df = pd.read_sql_query("SELECT * FROM nslib_book",conn)
book_df.head()
#테이블의 열 이름을 데이터프레임의 열 이름으로 자동으로 사용해줌

,name,author,borrow_count
0,인공지능과 흙,김동훈 지음,0
1,가짜 행복 권하는 사회,김태형 지음,0
2,나도 한 문장 잘 쓰면 바랄 게 없겠네,김선영 지음,0
3,예루살렘 해변,"이도 게펜 지음, 임재희 옮김",0
4,김성곤의 중국한시기행 : 장강·황하 편,김성곤 지음,0
